In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM , Dense
import numpy as np

batch_size = 64 
epochs = 50
latent_dim = 256
num_samples = 8000

input_texts = []
target_texts = []
input_char = set()
target_char = set()
with open('fra.txt','r',encoding = 'utf-8') as f:
  lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines)-1)]:
  input_text, target_text, _ =  line.split('\t')
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for x in input_text:
    if x not in input_char:
      input_char.add(x)
  for y in target_text:
    if y not in target_char:
      target_char.add(y)



In [ ]:
input_char = sorted(list(input_char))
target_char = sorted(list(target_char))
size_encoder_tokens = len(input_char)
size_decoder_tokens = len(target_char)
max_encoder_sequential_length = max([len(y) for y in input_texts])
max_decoder_sequential_length = max([len(x) for x in target_texts])

In [ ]:
print(' input character unique = ', input_char)
print(' target character unique = ', target_char)
print(' size of the unique character in input English = ', size_encoder_tokens)
print(' size of the unique character in target French = ', size_decoder_tokens)
print ( " maximum size of the sentence in input (English) = ", max_encoder_sequential_length)
print ( " maximum size of the sentence in output (French) = ", max_decoder_sequential_length)

 input character unique =  [' ', '!', '"', '$', '%', '&', "'", ',', '-', '.', '0', '1', '3', '5', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
 target character unique =  ['\t', '\n', ' ', '!', '%', '&', "'", '(', ')', ',', '-', '.', '0', '1', '3', '5', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '«', '»', 'À', 'Ç', 'É', 'Ê', 'à', 'â', 'ç', 'è', 'é', 'ê', 'î', 'ï', 'ô', 'ù', 'û', 'œ', '\u2009', '’', '\u202f']
 size of the unique character in input English =  69
 size of the unique character in target French =  92
 maximum size of 

In [ ]:
input_token_index = dict([(char,i) for i, char in enumerate(input_char)])
target_token_index = dict([(char,i) for i,char in enumerate(target_char)])

In [ ]:
# input_token_index
# target_token_index

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_sequential_length, size_encoder_tokens), dtype = "float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_sequential_length, size_decoder_tokens), dtype = "float32"
)
decoder_target_data = np.zeros(
     (len(input_texts), max_decoder_sequential_length, size_decoder_tokens), dtype = "float32"
)

In [ ]:
for i, (input_text,target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i,t,input_token_index[char]] = 1
  encoder_input_data[i,t+1:,input_token_index[' ']] = 1 
  for t,char in enumerate(target_text):
    decoder_input_data[i,t,target_token_index[char]] = 1
    if t > 0:
      decoder_target_data[i,t-1,target_token_index[char]] = 1

  decoder_input_data[i,t+1:,target_token_index[' ']] = 1
  decoder_target_data[i,t:,target_token_index[' ']] = 1

In [ ]:
encoder_inputs = Input(shape=(None, size_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(None, size_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(size_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics = ['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/50
100/100 [==============================] - 9s 20ms/step - loss: 1.1590 - accuracy: 0.7408 - val_loss: 1.0137 - val_accuracy: 0.7234
Epoch 2/50
100/100 [==============================] - 1s 11ms/step - loss: 0.8486 - accuracy: 0.7706 - val_loss: 0.8663 - val_accuracy: 0.7573
Epoch 3/50
100/100 [==============================] - 1s 11ms/step - loss: 0.7024 - accuracy: 0.8088 - val_loss: 0.7250 - val_accuracy: 0.7953
Epoch 4/50
100/100 [==============================] - 1s 11ms/step - loss: 0.5917 - accuracy: 0.8311 - val_loss: 0.6510 - val_accuracy: 0.8107
Epoch 5/50
100/100 [==============================] - 1s 11ms/step - loss: 0.5359 - accuracy: 0.8434 - val_loss: 0.6113 - val_accuracy: 0.8194
Epoch 6/50
100/100 [==============================] - 1s 11ms/step - loss: 0.4984 - accuracy: 0.8535 - val_loss: 0.5850 - val_accuracy: 0.8281
Epoch 7/50
100/100 [==============================] - 1s 12ms/step - loss: 0.4700 - accuracy: 0.8616 - val_loss: 0.5590 - val_accuracy: 0.8362